In [ ]:
## METHOD 1 - WEIGHTED LINEAR COMBINATION

##Calculate A Score based on the number of parcels within 1km, and their distances.

def Calculate_Score(filepath, fields):   
    ls = [[0 for j in range(0)] for i in range(4000000)]    ##list to store near table rows

    with arcpy.da.SearchCursor(filepath, fields) as cursor: ##iterate through near table and append rows to ls
        for row in cursor:
            ls[row[1]].append(row)
    
    ls2 = [[0 for j in range(0)] for i in range(4000000)]   ## 2D list to separate each set of near transit points per parcel
    count = 0
    for i in ls:                                            ## Append all rows for each parcel into separate lists
        for j in i:
            row = list(j)
            ls2[count].append(row)
        count+=1       

    for i in ls2:
        sum1=0
        for j in i:
            val = .5*(j[6]) + .2*(j[5]) + .3*(j[7])     ## Calculate Score based on bus, lightrail, highfreq
            dist = j[3]
            if dist >= 0 and dist < 200:                  ## Calculate distance penalty
                dist = 1
            elif dist >= 200 and dist < 400:
                dist = .7
            elif dist >= 400 and dist < 600:
                dist = .5
            elif dist >= 600 and dist < 800:
                dist = .5
            elif dist >= 800 and dist <= 1000:
                dist = .5
            else:
                dist = 0
            sum1 += (val * dist)                        ## Mulitply by distance penalty, sum all values to obtain final score
        
        for j in i:
            j[10] = sum1                                ## Add final score value to each cell
            
    ls4 = []
    for i in ls2:                                       ## Append all rows from nested list into 1 dimension list
        for j in i:
            ls4.append(j)        
    i=0
    with arcpy.da.UpdateCursor(filepath, fields) as cursor:  ## Read one dimension list and update parcel table with score
        for row in cursor:
            row[10] = ls4[i][10]
            cursor.updateRow(row)
            i+=1
                

In [ ]:
#Running sensitivity analysis

##Create copy of Hennepin Parcels
arcpy.management.Copy(r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Parcels\ParcelsHennepin.shp", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\ParcelsHennepin.shp", "ShapeFile", None)

##Select Sample points
arcpy.management.SelectLayerByAttribute("ParcelsHennepin2", "NEW_SELECTION", "VIEWID = 3247867 Or VIEWID = 3232719 Or VIEWID = 3210042 Or VIEWID = 3401444 Or VIEWID = 3401296 Or VIEWID = 3203364 Or VIEWID = 3202665 Or VIEWID = 3199454 Or VIEWID = 3397620 Or VIEWID = 3395224 Or VIEWID = 3195280 Or VIEWID = 3138642 Or VIEWID = 3076498 Or VIEWID = 3025724 Or VIEWID = 3097367 Or VIEWID = 3111651 Or VIEWID = 3018859 Or VIEWID = 3336795 Or VIEWID = 3002733 Or VIEWID = 3002194 Or VIEWID = 3125974 Or VIEWID = 3404010 Or VIEWID = 3226412 Or VIEWID = 3404203 Or VIEWID = 3274767 Or VIEWID = 3275026 Or VIEWID = 3271712 Or VIEWID = 3412531 Or VIEWID = 3306256 Or VIEWID = 3436868 Or VIEWID = 3248725 Or VIEWID = 3001890 Or VIEWID = 3162606 Or VIEWID = 3368601 Or VIEWID = 3214765 Or VIEWID = 3193607 Or VIEWID = 3197673 Or VIEWID = 3136243 Or VIEWID = 3299227 Or VIEWID = 3298765 Or VIEWID = 3022470 Or VIEWID = 3009490 Or VIEWID = 3010132 Or VIEWID = 3409679 Or VIEWID = 3239566 Or VIEWID = 3409587 Or VIEWID = 3248426 Or VIEWID = 3434324 Or VIEWID = 3306548 Or VIEWID = 3302126", "INVERT")
arcpy.management.DeleteRows("ParcelsHennepin2")

##Generate near table for sample points and join 
arcpy.analysis.GenerateNearTable("ParcelsHennepin2", "TransitStops_SpatialJoin", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsHennepin_GenerateNearTab2", "1 Kilometers", "LOCATION", "NO_ANGLE", "ALL", 45, "PLANAR")
arcpy.management.CalculateField("ParcelsHennepin_GenerateNearTab2", "Score", "0", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.JoinField("ParcelsHennepin_GenerateNearTab2", "NEAR_FID", "TransitStops_SpatialJoin", "OBJECTID", "Bus;Lightrail;HighFreq;POINT_X;POINT_Y", "NOT_USE_FM", None)

In [ ]:
##Running sensitivity analysis

##Generate near table for sample points and join 
arcpy.analysis.GenerateNearTable("ParcelsHennepin2", "TransitStops_SpatialJoin", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsHennepin_GenerateNearTab2", "1 Kilometers", "LOCATION", "NO_ANGLE", "ALL", 45, "PLANAR")
arcpy.management.CalculateField("ParcelsHennepin_GenerateNearTab2", "Score", "0", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.JoinField("ParcelsHennepin_GenerateNearTab2", "NEAR_FID", "TransitStops_SpatialJoin", "OBJECTID", "Bus;Lightrail;HighFreq;POINT_X;POINT_Y", "NOT_USE_FM", None)

fields = ["OBJECTID", "IN_FID", "NEAR_FID", "NEAR_DIST", "NEAR_RANK", "Bus", "Lightrail", "HighFreq", "POINT_X", "POINT_Y", "Score"]
filepath3 = r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsHennepin_GenerateNearTab2"



## Calculate Score for sample points and then join with accuracy assessment data
## Adjust model weights/parameters in Calculate_Score cell and re-run this cell 

Calculate_Score(filepath3, fields)
arcpy.management.CalculateField("ParcelsHennepin2", "Score", "0", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField("ParcelsHennepin2", "ID2", "!FID!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")
arcpy.management.JoinField("ParcelsHennepin2", "FID", "ParcelsHennepin_GenerateNearTab2", "IN_FID", "Score", "NOT_USE_FM", None)

arcpy.management.JoinField("ParcelsHennepin2", "VIEWID", "Transit1_TestData", "ID", "Actual_Score", "NOT_USE_FM", None)

##Convert table with scores to Excel table
arcpy.conversion.TableToExcel("ParcelsHennepin2", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\ParcelsHennepin2_TableToExcel.xlsx", "NAME", "CODE")

In [ ]:
##Final Model

##Generate Near Tables and join with Transit stops shapefile to obtain transit stop attributes for calculating score.

##Scott County
arcpy.analysis.GenerateNearTable("ParcelsScott", "TransitStops_SpatialJoin", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsScott_GenerateNearTab1", "1 Kilometers", "LOCATION", "NO_ANGLE", "ALL", 45, "PLANAR")
arcpy.management.CalculateField("ParcelsScott_GenerateNearTab", "Score", "0", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.JoinField("ParcelsScott_GenerateNearTab1", "NEAR_FID", "TransitStops_SpatialJoin", "OBJECTID", "Bus;Lightrail;HighFreq;POINT_X;POINT_Y", "NOT_USE_FM", None)

##Anoka
arcpy.analysis.GenerateNearTable("ParcelsAnoka", "TransitStops_SpatialJoin", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsAnoka_GenerateNearTab", "1 Kilometers", "LOCATION", "NO_ANGLE", "ALL", 45, "PLANAR")
arcpy.management.CalculateField("ParcelsAnoka_GenerateNearTab", "Score", "0", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.JoinField("ParcelsAnoka_GenerateNearTab", "NEAR_FID", "TransitStops_SpatialJoin", "OBJECTID", "Bus;Lightrail;HighFreq;POINT_X;POINT_Y", "NOT_USE_FM", None)

##Hennepin
arcpy.analysis.GenerateNearTable("ParcelsHennepin", "TransitStops_SpatialJoin", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsHennepin_GenerateNearTab", "1 Kilometers", "LOCATION", "NO_ANGLE", "ALL", 45, "PLANAR")
arcpy.management.CalculateField("ParcelsHennepin_GenerateNearTab", "Score", "0", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.JoinField("ParcelsHennepin_GenerateNearTab", "NEAR_FID", "TransitStops_SpatialJoin", "OBJECTID", "Bus;Lightrail;HighFreq;POINT_X;POINT_Y", "NOT_USE_FM", None)

##Carver
arcpy.analysis.GenerateNearTable("ParcelsCarver", "TransitStops_SpatialJoin", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsCarver_GenerateNearTab", "1 Kilometers", "LOCATION", "NO_ANGLE", "ALL", 45, "PLANAR")
arcpy.management.CalculateField("ParcelsCarver_GenerateNearTab", "Score", "0", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.JoinField("ParcelsCarver_GenerateNearTab", "NEAR_FID", "TransitStops_SpatialJoin", "OBJECTID", "Bus;Lightrail;HighFreq;POINT_X;POINT_Y", "NOT_USE_FM", None)

##Washington
arcpy.analysis.GenerateNearTable("ParcelsWashington", "TransitStops_SpatialJoin", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsWashington_GenerateNearTab", "1 Kilometers", "LOCATION", "NO_ANGLE", "ALL", 45, "PLANAR")
arcpy.management.CalculateField("ParcelsWashington_GenerateNearTab", "Score", "0", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.JoinField("ParcelsWashington_GenerateNearTab", "NEAR_FID", "TransitStops_SpatialJoin", "OBJECTID", "Bus;Lightrail;HighFreq;POINT_X;POINT_Y", "NOT_USE_FM", None)

##Ramsey
arcpy.analysis.GenerateNearTable("ParcelsRamsey", "TransitStops_SpatialJoin", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsRamsey_GenerateNearTab", "1 Kilometers", "LOCATION", "NO_ANGLE", "ALL", 45, "PLANAR")
arcpy.management.CalculateField("ParcelsRamsey_GenerateNearTab", "Score", "0", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.JoinField("ParcelsRamsey_GenerateNearTab", "NEAR_FID", "TransitStops_SpatialJoin", "OBJECTID", "Bus;Lightrail;HighFreq;POINT_X;POINT_Y", "NOT_USE_FM", None)

##Dakota
arcpy.analysis.GenerateNearTable("parcels_Dakota", "TransitStops_SpatialJoin", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsDakota_GenerateNearTab", "1 Kilometers", "LOCATION", "NO_ANGLE", "ALL", 45, "PLANAR")
arcpy.management.CalculateField("ParcelsDakota_GenerateNearTab", "Score", "0", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")
arcpy.management.JoinField("ParcelsDakota_GenerateNearTab", "NEAR_FID", "TransitStops_SpatialJoin", "OBJECTID", "Bus;Lightrail;HighFreq;POINT_X;POINT_Y", "NOT_USE_FM", None)

In [ ]:
##Run Calculate Score on each parcels near table, and then join the near table to the original parcel datasets
## Select all parcels that have NULL value for score, then set score to 0

filepath1 = r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsScott_GenerateNearTab1"
filepath2 = r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsAnoka_GenerateNearTab"
filepath3 = r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsHennepin_GenerateNearTab"
filepath4 = r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsCarver_GenerateNearTab"
filepath5 = r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsWashington_GenerateNearTab"
filepath6 = r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsRamsey_GenerateNearTab"
filepath7 = r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\ParcelsDakota_GenerateNearTab"
fields = ["OBJECTID", "IN_FID", "NEAR_FID", "NEAR_DIST", "NEAR_RANK", "Bus", "Lightrail", "HighFreq", "POINT_X", "POINT_Y", "Score"]

Calculate_Score(filepath1, fields)
arcpy.management.JoinField("ParcelsScott", "FID", "ParcelsScott_GenerateNearTab", "IN_FID", "Score", "NOT_USE_FM", None)
arcpy.management.SelectLayerByAttribute("ParcelsScott", "NEW_SELECTION", "Score IS NULL", None)
arcpy.management.CalculateField("ParcelsScott", "Score", "0", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

Calculate_Score(filepath2, fields)
arcpy.management.JoinField("ParcelsAnoka", "FID", "ParcelsAnoka_GenerateNearTab", "IN_FID", "Score", "NOT_USE_FM", None)
arcpy.management.SelectLayerByAttribute("ParcelsAnoka", "NEW_SELECTION", "Score IS NULL", None)
arcpy.management.CalculateField("ParcelsAnoka", "Score", "0", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

Calculate_Score(filepath3, fields)
arcpy.management.JoinField("ParcelsHennepin", "FID", "ParcelsHennepin_GenerateNearTab", "IN_FID", "Score", "NOT_USE_FM", None)
arcpy.management.SelectLayerByAttribute("ParcelsHennepin", "NEW_SELECTION", "Score IS NULL", None)
arcpy.management.CalculateField("ParcelsHennepin", "Score", "0", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

Calculate_Score(filepath4, fields)
arcpy.management.JoinField("ParcelsCarver", "FID", "ParcelsCarver_GenerateNearTab", "IN_FID", "Score", "NOT_USE_FM", None)
arcpy.management.SelectLayerByAttribute("ParcelsCarver", "NEW_SELECTION", "Score IS NULL", None)
arcpy.management.CalculateField("ParcelsCarver", "Score", "0", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

Calculate_Score(filepath5, fields)
arcpy.management.JoinField("ParcelsWashington", "FID", "ParcelsWashington_GenerateNearTab", "IN_FID", "Score", "NOT_USE_FM", None)
arcpy.management.SelectLayerByAttribute("ParcelsWashington", "NEW_SELECTION", "Score IS NULL", None)
arcpy.management.CalculateField("ParcelsWashington", "Score", "0", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

Calculate_Score(filepath6, fields)
arcpy.management.JoinField("ParcelsRamsey", "FID", "ParcelsRamsey_GenerateNearTab", "IN_FID", "Score", "NOT_USE_FM", None)
arcpy.management.SelectLayerByAttribute("ParcelsRamsey", "NEW_SELECTION", "Score IS NULL", None)
arcpy.management.CalculateField("ParcelsRamsey", "Score", "0", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

Calculate_Score(filepath7, fields)
arcpy.management.JoinField("parcels_Dakota", "OBJECTID", "ParcelsDakota_GenerateNearTab", "IN_FID", "Score", "NOT_USE_FM", None)
arcpy.management.SelectLayerByAttribute("parcels_Dakota", "NEW_SELECTION", "Score IS NULL", None)
arcpy.management.CalculateField("parcels_Dakota", "Score", "0", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")


In [ ]:
## Deselect all selections from each parcel datset before running

## Merge all parcels into one shape
arcpy.management.Merge("parcels_Dakota;ParcelsAnoka;ParcelsCarver;ParcelsHennepin;ParcelsRamsey;ParcelsScott;ParcelsWashington", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\parcels_Merge", 'COUNTY_ID "COUNTY_ID" true true false 3 Text 0 0,First,#,parcels_Dakota,COUNTY_ID,0,3;Shape_Area "Shape_Area" true true false 19 Double 0 0,First,#,parcels_Dakota,Shape_Area,-1,-1,ParcelsAnoka,Shape_Area,-1,-1,ParcelsCarver,Shape_Area,-1,-1,ParcelsHennepin,Shape_Area,-1,-1,ParcelsRamsey,Shape_Area,-1,-1,ParcelsScott,Shape_Area,-1,-1,ParcelsWashington,Shape_Area,-1,-1;NEAR_FID "NEAR_FID" true true false 10 Long 0 10,First,#,parcels_Dakota,NEAR_FID,-1,-1,ParcelsAnoka,NEAR_FID,-1,-1,ParcelsCarver,NEAR_FID,-1,-1,ParcelsHennepin,NEAR_FID,-1,-1,ParcelsRamsey,NEAR_FID,-1,-1,ParcelsWashington,NEAR_FID,-1,-1;NEAR_DIST "NEAR_DIST" true true false 19 Double 0 0,First,#,parcels_Dakota,NEAR_DIST,-1,-1,ParcelsAnoka,NEAR_DIST,-1,-1,ParcelsCarver,NEAR_DIST,-1,-1,ParcelsHennepin,NEAR_DIST,-1,-1,ParcelsRamsey,NEAR_DIST,-1,-1,ParcelsWashington,NEAR_DIST,-1,-1;ID2 "ID2" true true false 10 Long 0 10,First,#,parcels_Dakota,ID2,-1,-1,ParcelsAnoka,ID2,-1,-1,ParcelsCarver,ID2,-1,-1,ParcelsHennepin,ID2,-1,-1,ParcelsRamsey,ID2,-1,-1,ParcelsScott,ID2,-1,-1,ParcelsWashington,ID2,-1,-1;Score "Score" true true false 13 Float 0 0,First,#,parcels_Dakota,Score,-1,-1,ParcelsAnoka,Score,-1,-1,ParcelsCarver,Score,-1,-1,ParcelsHennepin,Score,-1,-1,ParcelsRamsey,Score,-1,-1,ParcelsScott,Score,-1,-1,ParcelsWashington,Score,-1,-1', "NO_SOURCE_INFO")

## Standardize Score from 0 to 1
arcpy.management.CalculateField("parcels_Merge", "STD_score", "(!Score!) / (16.3)", "PYTHON3", '', "FLOAT", "NO_ENFORCE_DOMAINS")